In [1]:
import pandas as pd
import requests
import formula1 as f1

In [2]:
# Accessing the Ergast F1 API. This was especially complex and could only be done in python due to the multiple levels 
# of nesting (MRData, StandingsTable, StandingsLists etc) present in the API
# Inspired by the pyErgast python module

def constructor_standings(year):
    url = 'http://ergast.com/api/f1/{}/constructorStandings.json?limit=1000000'.format(year)
    r = requests.get(url)
    constructorStandings = r.json()['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']
    for i in constructorStandings:
        i['constructorID'] = i['Constructor']['constructorId']
        i['name'] = i['Constructor']['name']
        del i['Constructor']
    return pd.DataFrame(constructorStandings)

In [3]:
# For each year between 2012 and 2021 inclusive, obtain the constructor standings and create a master dataframe with all datapoints (only name, points and a unique constructor id)
master_data = pd.DataFrame()

for i in range(2012, 2022):
    df = constructor_standings(i)
    df = df[['name', 'points', 'constructorID']]
    df['year'] = i
    master_data = master_data.append(df)


In [4]:
# List of constructors that we need
constructor_list = ['mercedes', 'red_bull', 'ferrari', 'williams', 'mclaren', 'alpine', 'alphatauri', 'alfa', 'aston_martin']

In [5]:
# Replacing the teams which had name/ownership changes to the current name to match up with R&D collection methodology

master_data['constructorID'].replace("lotus_f1", "alpine", inplace=True)
master_data['constructorID'].replace("renault", "alpine", inplace=True)

master_data['constructorID'].replace("sauber", "alfa", inplace=True)

master_data['constructorID'].replace("toro_rosso", "alphatauri", inplace=True)

master_data['constructorID'].replace("force_india", "aston_martin", inplace=True)
master_data['constructorID'].replace("racing_point", "aston_martin", inplace=True)

In [6]:
master_data['name'].replace("Lotus F1", "Alpine F1 Team", inplace=True)
master_data['name'].replace("Renault", "Alpine F1 Team", inplace=True)

master_data['name'].replace("Sauber", "Alfa Romeo", inplace=True)

master_data['name'].replace("Toro Rosso", "AlphaTauri", inplace=True)

master_data['name'].replace("Force India", "Aston Martin", inplace=True)
master_data['name'].replace("Racing Point", "Aston Martin", inplace=True)

In [7]:
# Only keep the 9 teams being analysed 
master_data = master_data[master_data['constructorID'].isin(constructor_list)]

In [8]:
# Removing 2021 for the regressions, a different CSV file was created including the 2021 data for the section 2 graph
master_data.reset_index(drop=True)
master_data = master_data[master_data['year'] != 2021]

The following section calculates the number of points scored by all teams in a given year, and then finds the proportion of that total that each team scored

In [16]:
master_data['points'] = master_data['points'].astype(int)

In [19]:
def categorise(row):
    return master_data.loc[master_data['year']==row['year'], 'points'].sum()
master_data['totalpoints'] = master_data.apply(lambda row: categorise(row), axis=1)

In [23]:
master_data['proptotalpoints'] = (master_data['points']/master_data['totalpoints'])

In [37]:
master_data.to_csv('constructor-standings-2012-2021-proportions.csv')